# User Recommendation System

## Data

In [123]:
# Importing necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [124]:
linkedin=pd.read_csv("linkedin.csv",index_col=False)

In [125]:
linkedin.head(2)

,index,category,linkedin,profile_picture,description,Experience,Name,position,location,skills,clean_skills
0,1,HR,https://in.linkedin.com/in/sameer-wadhawan-b55...,https://media-exp1.licdn.com/dms/image/C5603AQ...,"An experienced HR professional, HR mentor and...",Senior Vice President & Head of HRCompany Name...,Sameer Wadhawan,Senior Vice President and Head of HR- Samsung ...,"Gurgaon, Haryana, India","['\nPerformance Management\n', '\nHuman Resour...","['Performance Management', 'Human Resources', ..."
1,2,HR,https://in.linkedin.com/in/adarsh-krishna-a4ab0a5,https://media-exp1.licdn.com/dms/image/C5103AQ...,"Head Talent Acquisition, HR Leader and Strateg...",Head of Talent Acquisition - India & APAC and ...,Adarsh Krishna,Head Talent Acquisition and HR Leader for Heal...,"Pune, Maharashtra, India","['\nTalent Acquisition\n', '\nEmployee Engagem...","['Talent Acquisition', 'Employee Engagement', ..."


In [126]:
linkedin.shape

(1251, 11)

In [127]:
linkedin= linkedin.drop('profile_picture', axis=1)
linkedin= linkedin.drop('linkedin', axis=1)
linkedin= linkedin.drop('skills', axis=1)

In [128]:
linkedin.head(2)

,index,category,description,Experience,Name,position,location,clean_skills
0,1,HR,"An experienced HR professional, HR mentor and...",Senior Vice President & Head of HRCompany Name...,Sameer Wadhawan,Senior Vice President and Head of HR- Samsung ...,"Gurgaon, Haryana, India","['Performance Management', 'Human Resources', ..."
1,2,HR,"Head Talent Acquisition, HR Leader and Strateg...",Head of Talent Acquisition - India & APAC and ...,Adarsh Krishna,Head Talent Acquisition and HR Leader for Heal...,"Pune, Maharashtra, India","['Talent Acquisition', 'Employee Engagement', ..."


In [129]:
linkedin=linkedin.rename(columns={"clean_skills":'skills',
                                  "category":"industry",
                                  "position":"jobTitle",
                                  "index":"userId"})

In [130]:
emptyexp = linkedin['Experience'].isnull().sum()
print(emptyexp)
emptylocation = linkedin['location'].isnull().sum()
print(emptylocation)

25
12


In [131]:
linkedin.dropna(subset=['Experience'], inplace=True)
emptyexp = linkedin['Experience'].isnull().sum()
print(emptyexp)

0


In [132]:
linkedin = linkedin[~linkedin['location'].str.contains('connections', case=False)]

linkedin.dropna(subset=['location'], inplace=True)
emptylocation = linkedin['location'].isnull().sum()
print(emptylocation)

0


In [133]:
linkedin.shape

(1216, 8)

In [134]:
linkedin.head(1)

,userId,industry,description,Experience,Name,jobTitle,location,skills
0,1,HR,"An experienced HR professional, HR mentor and...",Senior Vice President & Head of HRCompany Name...,Sameer Wadhawan,Senior Vice President and Head of HR- Samsung ...,"Gurgaon, Haryana, India","['Performance Management', 'Human Resources', ..."


In [135]:
emptyskills = linkedin['skills'].apply(lambda x: x == [""]).sum()
print(emptyskills)

0


In [136]:
#change location to store city and country:
cities = []
countries = []

# Loop through each location in the DataFrame
for location in linkedin['location']:
    location_parts = location.split(', ')
    if len(location_parts) > 1:
        country = location_parts[-1]
        city = location_parts[0]
    else:
        # If there's only one part, consider it as the country
        country = location_parts[0]
        city = ""  # Set city to an empty string
    cities.append(city)
    countries.append(country)

# Add new columns for city and country to the DataFrame
linkedin['City'] = cities
linkedin['Country'] = countries

linkedin= linkedin.drop('location', axis=1)

# Display the updated DataFrame
print(linkedin)


      userId  industry                                        description  \
0          1        HR  An experienced HR professional,  HR mentor and...   
1          2        HR  Head Talent Acquisition, HR Leader and Strateg...   
2          3        HR  A Talent Acquisition and HR professional with ...   
3          4        HR                                                NaN   
4          5        HR  Over 18 Years of experience in IT /ITES  / BPO...   
...      ...       ...                                                ...   
1246    1753  Aviation                                                NaN   
1247    1754  Aviation                                                NaN   
1248    1755  Aviation  An MBA Graduate specialized in Aviation Manage...   
1249    1756  Aviation                                                NaN   
1250    1757  Aviation  Currently I'm pursuing MBA in Aviation Managem...   

                                             Experience  \
0     Senior Vic

In [137]:
linkedin.dtypes

userId          int64
industry       object
description    object
Experience     object
Name           object
jobTitle       object
skills         object
City           object
Country        object
dtype: object

In [138]:

# Standardize job titles by converting them to lowercase and removing leading/trailing white spaces
linkedin['jobTitle'] = linkedin['jobTitle'].str.lower().str.strip()

# Standardize skills by converting them to lowercase and removing leading/trailing white spaces
linkedin['skills'] = linkedin['skills'].str.lower().str.strip()

# Standardize industry names by converting them to lowercase and removing leading/trailing white spaces
linkedin['industry'] = linkedin['industry'].str.lower().str.strip()

# Standardize city names by converting them to lowercase and removing leading/trailing white spaces
linkedin['City'] = linkedin['City'].str.lower().str.strip()

# Standardize country names by converting them to lowercase and removing leading/trailing white spaces
linkedin['Country'] = linkedin['Country'].str.lower().str.strip()



# Create a mapping dictionary to map specific locations to "United States"
location_mapping = {
    'oklahoma area': 'united states',
    'oklahoma': 'united states',
    'greater denver area': 'united states',
    'connecticut': 'united states',
    'pennsylvania area': 'united states',
    'virginia area': 'united states',
    'new mexico': 'united states',
    'michigan': 'united states',
    'dallas/fort worth area': 'united states',
    'greater philadelphia area': 'united states',
    'washington d.c. metro area': 'united states',
    'california area': 'united states',
    'florida area': 'united states',
    'ohio': 'united states',
    'greater minneapolis-st. paul area': 'united states',
    'cote d’ivoire (ivory coast)': 'cote d’ivoire (ivory coast)',  # Leave as is
    'arizona': 'united states',
    'louisiana': 'united states',
    'washington': 'united states',
    'greater san diego area': 'united states',
    'tennessee area': 'united states',
    'maryland area': 'united states',
    'oregon': 'united states',
    'texas area': 'united states',
    'iowa area': 'united states',
    'maryland': 'united states',
    'wisconsin': 'united states',
    'greater atlanta area': 'united states',
    'san francisco bay area': 'united states',
    'new jersey': 'united states',
    'missouri': 'united states',
    'greater boston area': 'united states',
    'new york': 'united states',
    'florida': 'united states',
    'pennsylvania': 'united states',
    'indiana area': 'united states',
    'north carolina area': 'united states',
    'greater milwaukee area': 'united states',
    'north carolina': 'united states',
    'greater chicago area': 'united states',
    'district of columbia': 'united states',
    'massachusetts': 'united states',
    'greater new york city area': 'united states',
    'texas': 'united states',
    'illinois': 'united states',
    'greater los angeles area': 'united states',
    'california': 'united states',
    "canada area": 'canada',
}

default_mapping = {Country: Country for Country in linkedin['Country'].unique()}

# Apply the combined mapping
linkedin['Country'] = linkedin['Country'].map(location_mapping).fillna(linkedin['Country'].map(default_mapping))

In [139]:
unique_counts = linkedin.nunique()
print(unique_counts)

userId         1216
industry         25
description     632
Experience     1187
Name           1187
jobTitle       1147
skills          763
City            298
Country          37
dtype: int64


In [140]:
#get unique values

attributes = ['industry','Country']

for attribute in attributes:
    unique_values = linkedin[attribute].unique()
    print(f"Unique values for {attribute}:")
    for value in unique_values:
        print(value)
    print("\n")


Unique values for industry:
hr
designing
managment
information technology
education
advocate
business development
health & fitness
agricultural
bpo
sales
consultant
digital media
building & construction
automobile
banking
engineering
food & beverages
finance
apparel
accountant
architects
public relations
arts
aviation


Unique values for Country:
india
canada
united states
togo
australia
djibouti
saudi arabia
united kingdom
korea
united arab emirates
hong kong
singapore
ireland
israel
libya
ethiopia
nigeria
cote d’ivoire (ivory coast)
georgia
seychelles
grenada
brunei darussalam
gambia
hungary
luxembourg
cameroon
sri lanka
angola
myanmar
kenya
rwanda
monaco area
mongolia
isle of man
et
jordan
philippines




In [141]:
#change skills to an array of strings not one big 
def skills_to_list(skills_string):
    return ast.literal_eval(skills_string)

linkedin['skills'] = linkedin['skills'].apply(skills_to_list)

In [142]:
overall_skill_count = linkedin['skills'].apply(lambda skills: len(skills)).sum()
print("Overall number of skills:", overall_skill_count)

Overall number of skills: 9016


In [143]:
all_skills = [skill for skills in linkedin['skills'] for skill in skills]
unique_skills_count = len(set(all_skills))

print("Number of unique skills:", unique_skills_count)

Number of unique skills: 2191


In [144]:
linkedin.head(1)

,userId,industry,description,Experience,Name,jobTitle,skills,City,Country
0,1,hr,"An experienced HR professional, HR mentor and...",Senior Vice President & Head of HRCompany Name...,Sameer Wadhawan,senior vice president and head of hr- samsung ...,"[performance management, human resources, tale...",gurgaon,india


In [145]:
print(type(linkedin['skills'][0][0]))

<class 'str'>


In [146]:
empty_skills = linkedin['skills'].apply(lambda skills: all(skill.strip() == '' or skill.strip() == '""' or skill.isspace() for skill in skills))
num_empty_skills = empty_skills.sum()
print("Number of users with empty skills:", num_empty_skills)

Number of users with empty skills: 415


In [147]:
# Filter and print the users with empty skills to make sure 
#if only one of the skills is empty or the entire array
users_with_empty_skills = linkedin[empty_skills]
print("Users with empty skills:")
for index, user in users_with_empty_skills.iterrows():
    print("User:", index)
    print("Skills:", user['skills'])
    print()

Users with empty skills:
User: 3
Skills: ['']

User: 14
Skills: ['']

User: 16
Skills: ['']

User: 25
Skills: ['']

User: 29
Skills: ['']

User: 30
Skills: ['']

User: 31
Skills: ['']

User: 35
Skills: ['']

User: 43
Skills: ['']

User: 48
Skills: ['']

User: 53
Skills: ['']

User: 61
Skills: ['']

User: 64
Skills: ['']

User: 67
Skills: ['']

User: 69
Skills: ['']

User: 71
Skills: ['']

User: 78
Skills: ['']

User: 81
Skills: ['']

User: 85
Skills: ['']

User: 88
Skills: ['']

User: 89
Skills: ['']

User: 91
Skills: ['']

User: 92
Skills: ['']

User: 94
Skills: ['']

User: 97
Skills: ['']

User: 99
Skills: ['']

User: 100
Skills: ['']

User: 101
Skills: ['']

User: 102
Skills: ['']

User: 105
Skills: ['']

User: 106
Skills: ['']

User: 107
Skills: ['']

User: 108
Skills: ['']

User: 112
Skills: ['']

User: 115
Skills: ['']

User: 116
Skills: ['']

User: 117
Skills: ['']

User: 129
Skills: ['']

User: 134
Skills: ['']

User: 136
Skills: ['']

User: 138
Skills: ['']

User: 139
Skills: 

In [148]:
linkedin = linkedin[~linkedin['skills'].apply(lambda skills: all(skill.strip() == '' or skill.strip() == '""' or skill.isspace() for skill in skills))]

In [149]:
empty_skills = linkedin['skills'].apply(lambda skills: all(skill.strip() == '' or skill.strip() == '""' or skill.isspace() for skill in skills))
num_empty_skills = empty_skills.sum()
print("Number of users with empty skills:", num_empty_skills)

Number of users with empty skills: 0


In [150]:
print(linkedin)

      userId  industry                                        description  \
0          1        hr  An experienced HR professional,  HR mentor and...   
1          2        hr  Head Talent Acquisition, HR Leader and Strateg...   
2          3        hr  A Talent Acquisition and HR professional with ...   
4          5        hr  Over 18 Years of experience in IT /ITES  / BPO...   
5          6        hr  Human Resources Business Partner with a demons...   
...      ...       ...                                                ...   
1243    1750  aviation                                                NaN   
1244    1751  aviation  I am Ritu Roy,born in India. i am an energetic...   
1248    1755  aviation  An MBA Graduate specialized in Aviation Manage...   
1249    1756  aviation                                                NaN   
1250    1757  aviation  Currently I'm pursuing MBA in Aviation Managem...   

                                             Experience             Name  \

In [151]:
linkedin.reset_index(drop=True, inplace=True)

In [152]:
linkedin.shape

(801, 9)

In [153]:
# Shuffle the order of rows in the DataFrame
linkedin = linkedin.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the first few rows of the shuffled DataFrame
print(linkedin.head())

   userId    industry                                        description  \
0    1606  architects                                                NaN   
1    1558  architects  Experienced Owner with a demonstrated history ...   
2      85   designing                                                NaN   
3    1336  automobile  To work in a firm with a professional work dri...   
4      91   designing  An author, lecturer, teacher, application engi...   

                                          Experience                Name  \
0  Junior ArchitectCompany NameMETRICCHANT DESIGN...       Krati Bhambry   
1  OWNER/ FOUNDERCompany NameARCHITECT NAVED DESI...         Naved Patel   
2  Industrial Structures -Planning and DesigningC...        SUBODH BHATT   
3  EngineerCompany NameAUTOMOBILE ENGINEERING COM...  Vishnu Vishwakarma   
4  Founder, Author, TeacherCompany NameConcepts a...       Thom Tremblay   

                                            jobTitle  \
0                             

In [154]:
print(linkedin['userId'])

0      1606
1      1558
2        85
3      1336
4        91
       ... 
796     104
797     161
798     390
799    1161
800     157
Name: userId, Length: 801, dtype: int64


In [155]:
linkedin.reset_index(drop=True, inplace=True)

## Model

Content based Filtering:
TF-IDF + cosine Similarity:
You can use TF-IDF (Term Frequency-Inverse Document Frequency) to vectorize the industry and skills features and calculate the cosine similarity between user profiles. This approach can identify users with similar industry and skills profiles.

After trying multiple different models this was the most effective

In [156]:
#training model

def generate_recommendations(linkedin_data, top_k=5):
    # Combine industry, skills, and country into a single text feature for TF-IDF
    linkedin_data['profile'] = linkedin_data['industry'] + ' ' + linkedin_data['skills'].apply(lambda skills: ' '.join(skills))

    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the profile text into TF-IDF features
    tfidf_matrix = tfidf_vectorizer.fit_transform(linkedin_data['profile'])

    # Calculate cosine similarity between user profiles
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    def get_top_recommendations(user_index, cosine_sim, top_k=top_k):
        # Get the cosine similarity scores for the user
        sim_scores = list(enumerate(cosine_sim[user_index]))

        # Sort users based on similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the indices of top recommended users (excluding the user itself)
        top_recommendations = [i for i, _ in sim_scores if i != user_index][:top_k]

        return top_recommendations

    all_recommendations = {}

    for user_index in range(len(linkedin_data)):
        recommendations = get_top_recommendations(user_index, cosine_sim)

        # Convert indices to actual userIds
        user_id = linkedin_data.loc[user_index, 'userId']
        top_recommendation_user_ids = linkedin_data.loc[recommendations, 'userId'].tolist()

        all_recommendations[user_id] = top_recommendation_user_ids

    return all_recommendations

# Example Usage
result = generate_recommendations(linkedin, top_k=5)
print(result)


{1606: [1599, 1583, 1571, 1584, 1592], 1558: [1559, 1560, 1573, 1575, 1604], 85: [71, 234, 1611, 1461, 43], 1336: [1299, 1347, 1350, 1246, 1219], 91: [1132, 939, 320, 343, 975], 1498: [1488, 1496, 1469, 76, 1672], 949: [1206, 936, 306, 136, 1626], 1261: [1264, 1251, 1280, 1420, 1262], 1374: [1372, 1367, 1375, 1365, 1364], 1209: [1239, 1224, 1212, 1215, 1276], 88: [59, 1607, 1576, 51, 1592], 409: [125, 22, 278, 71, 123], 1518: [1524, 1510, 1542, 1541, 1513], 94: [1606, 60, 1739, 1599, 88], 927: [890, 894, 885, 905, 884], 1110: [1608, 1128, 1118, 1204, 1203], 356: [355, 360, 372, 387, 370], 1044: [1042, 1032, 1050, 216, 1512], 1464: [1460, 1469, 1459, 1463, 1496], 1322: [10, 8, 13, 1034, 1396], 1546: [1555, 1540, 1530, 1516, 1521], 1325: [1298, 1246, 1299, 1219, 1559], 1499: [1461, 1496, 1332, 1469, 1468], 1631: [1656, 1615, 1648, 1610, 1622], 972: [940, 978, 207, 953, 1705], 40: [34, 19, 33, 29, 6], 377: [395, 393, 371, 374, 370], 1520: [1524, 1537, 1541, 1530, 1515], 1449: [1224, 1212,

Since we dont have the actual recommendations or connections were going to use proxy metric
Proxy Metrics:

Even without actual user behavior, you can use proxy metrics to assess the quality of your recommendations. For example:
Coverage: Measure how many unique users are covered by your recommendations.
Diversity: Check the diversity of your recommendations across users.
Novelty: Evaluate how novel or unexpected your recommendations are.
do this on the above model

In [157]:
total_users = len(linkedin['userId'])
covered_users = len(set(user_id for recommendations in result.values() for user_id in recommendations))
coverage = covered_users / total_users * 100

print(f"Coverage: {coverage:.2f}%")

Coverage: 85.02%


In [158]:
all_industries = set(linkedin['industry'])
all_skills = set(skill for skills in linkedin['skills'] for skill in skills)

diversity_scores = []

for recommendations in result.values():
    recommended_industries = set(linkedin.loc[linkedin['userId'].isin(recommendations), 'industry'])
    recommended_skills = set(skill for skills in linkedin.loc[linkedin['userId'].isin(recommendations), 'skills'] for skill in skills)

    industry_diversity = len(recommended_industries) / len(all_industries)
    skill_diversity = len(recommended_skills) / len(all_skills)

    diversity_scores.append((industry_diversity + skill_diversity) / 2)

average_diversity = sum(diversity_scores) / len(diversity_scores)

print(f"Diversity: {average_diversity:.2f}")


Diversity: 0.05


In [159]:
def get_popularity_score(item):
    # For simplicity, using the number of times an item is recommended
    return np.sum(all_recommendations_array == item)


def calculate_novelty(recommendations_array):
    # Flatten the recommendations array
    all_recommendations_array = np.concatenate(recommendations_array)

    # Calculate popularity scores for each item
    popularity_scores = np.array([get_popularity_score(item) for item in np.unique(all_recommendations_array)])

    # Calculate novelty as the inverse of the average popularity score
    novelty_score = 1 / np.mean(popularity_scores)

    return novelty_score


all_recommendations_array = np.array(list(result.values()))

# Calculate novelty
novelty_score = calculate_novelty(all_recommendations_array)

print(f"Novelty Score: {novelty_score*100:.2f}%")

Novelty Score: 17.00%


In [160]:
#TESTING MODEL

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
    
# Fit and transform the profile text into TF-IDF features
tfidf_matrix = tfidf_vectorizer.fit_transform(linkedin['profile'])
    
# Calculate cosine similarity between user profiles
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
def get_top_recommendations(user_index,top_k=5):
    # Get the cosine similarity scores for the user
    sim_scores = list(enumerate(cosine_sim[user_index]))

    # Sort users based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of top recommended users (excluding the user itself)
    top_recommendations = [i for i, _ in sim_scores if i != user_index][:top_k]

    return top_recommendations

user=23
recommendations = get_top_recommendations(user)
print(f"Recommendations for userId: '{user}' are:\n{recommendations}")    

Recommendations for userId: '23' are:
[548, 96, 391, 715, 574]


In [161]:
import joblib

# Save the model
joblib.dump(get_top_recommendations, 'model.joblib')
# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
# Save the cosine similarity matrix
joblib.dump(cosine_sim, 'cosine_similarity_matrix.joblib')


['cosine_similarity_matrix.joblib']